In [1]:
from functools import wraps
import inspect
def Singleton(cls):
    assert inspect.isclass(cls), 'The Singleton decorator can only be applied to class'

    def run_only_once(func):
        """
         使__init__仅仅运行一次
        """

        @wraps(func)
        def _wrapper_init(self, *args, **kwargs):
            if not hasattr(self, "___init__called"):
                self.___init__called = True
                return func(self, *args, **kwargs)

        return _wrapper_init

    def newfunc(clsobj, *args, **kwargs):
        """
        使用元编程确保仅仅实现一个单例
        """
        if not hasattr(clsobj, "__instance"):
            setattr(clsobj, "__instance", object.__new__(clsobj))
        return getattr(clsobj, "__instance")

    # 拦截__new__方法
    setattr(cls, "__new__", classmethod(newfunc))
    # 由于在创建cls时，python会调用__new__来创建实例，然后再调用实例的__init__
    # 这样通过在__new__使用object__new__能确保每次实例化cls时均返回同一个实例。
    # 但是python还是会调用__new__的__init__，这样会导致__init__方法每实例一次就会调用一次
    # 这种行为不是我们需要的，因此我们通过run_on_once装饰器做一定的处理，使__init__只调用一次
    setattr(cls, "__init__", run_only_once(cls.__init__))
    return cls

In [2]:
@Singleton
class ABase:
    pass

class C(ABase):
    pass

class D(ABase):
    pass

In [3]:
c1 = C()
c2 = C()
d1 = D()
d2 = D()
print("id of c1 is %s" % id(c1))
print("id of c2 is %s" % id(c2))
print("C %s singleton" % ("is" if id(c1) == id(c2) else "isn't"))
print("id of d1 is %s" % id(d1))
print("id of d2 is %s" % id(d2))
print("D %s singleton" % ("is" if id(d1) == id(d2) else "isn't"))

id of c1 is 81921640
id of c2 is 81921640
C is singleton
id of d1 is 81921696
id of d2 is 81921696
D is singleton
